Question-1

Write a python program which searches all the product under a particular product from www.amazon.in. 
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search 
for guitars.

In [2]:
#Importing required files

import selenium
import warnings
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

term=input("Enter what you want to search : ")

#initiating driver


s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()

#linking Site

driver.get("https://www.amazon.in/")

#passing data in search field

items=driver.find_element(By.XPATH,"/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input")
items.send_keys(term)

#Clicking search button

search=driver.find_element(By.XPATH,"/html/body/div[1]/header/div/div[1]/div[2]/div[1]/form/div[3]/div/span/input")
search.click()

time.sleep(7)

Enter what you want to search :  laptop


Question-2

In the above question, now scrape the following details of each product listed in first 3 pages of your 
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
results then scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [4]:
product_replacement=[]
product_stock=[]
product_store=[]
product_price=[]
product_name=[]
product_rating=[]
product_time=[]
product_link=[]
product_brand=[]

stock=driver.find_element(By.XPATH,'//*[@id="p_n_availability/1318485031"]/span/a/span')

stock.click()
time.sleep(5)

start=0
end=3

for page in range(start,end):   
    
    try:
        items=driver.find_elements(By.XPATH,'//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-2"]')
        for i in items:
            name=i.text                                         #Fetching Names
            product_name.append(name)
    except NoSuchElementException:
        product_name.append("-")
    

#Fetching rating of product and creating array
    try:
        rating=driver.find_elements(By.XPATH,'//span[@class="a-size-base"]')
        for i in rating:
            rate=i.text
            product_rating.append(rate)
    except NoSuchElementException:
        product_rating.append("-")
    
    try:
        date=driver.find_elements(By.XPATH,'//span[@class="a-color-base a-text-bold"]')#for delivery date
        for i in date:
            product_time.append(i.text)
    except NoSuchElementException:
        product_time.append("-")
    
        
    link=driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    for i in link:
        tag=i.get_attribute("href")                 #Fetching Link of items
        product_link.append(tag)
    
    try:
        price=driver.find_elements(By.XPATH,'//span[@class="a-price-whole"]')
        for i in price:
            cost=i.text                                    #Fetching Price
            product_price.append(cost)
    except NoSuchElementException:
        product_price.append("-")
    
        
    next_page=driver.find_element(By.XPATH,"//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")
    next_page.click()

    time.sleep(4)
    
n=len(product_link)

#fetching other details by passing the URL of each product

for i in range(0,n):
    url=product_link[i]
    driver.get(url)
    
    time.sleep(4)
    
    try:
        replacement=driver.find_elements(By.XPATH,'//*[@id="RETURNS_POLICY"]/span/div[2]/a')
        for i in replacement:
            product_replacement.append(i.text)                     #Fetching Delivery time
    except NoSuchElemetException:
        product_replacement.append("-")
    
    try:
        instock=driver.find_elements(By.XPATH,'//span[@class="a-size-medium a-color-success"]')    #for stock check
        for i in instock:
            product_stock.append(i.text)                     #Fetching Delivery time
    except NoSuchElementException:
        product_stock.append("-")
        
    try:
        store=driver.find_elements(By.ID,"bylineInfo")   
        for i in store:
            product_store.append(i.text)                     
    except NoSuchElementException:
        product_store.append("-")





len(product_link)


48

In [30]:
#Creating data frame of data
df=pd.DataFrame({"Product Name " :product_name[:20],"Brand-store":product_store[:20],"Price ": product_price[:20],
                 "Ratings" : product_rating[:20],"Delivery-Time":product_time[:20],
                "Stock-Position": product_stock[:20],"Replacement-Time":product_replacement[:20],
                 "Link to Buy":product_link[:20]})
df

,Product Name,Brand-store,Price,Ratings,Delivery-Time,Stock-Position,Replacement-Time,Link to Buy
0,"HP ProBook 440 G8 Notebook PC, 11th Gen Intel ...",Visit the HP Store,"49,990",,"Tomorrow, December 26",In stock.,7 days Replacement,https://www.amazon.in/HP-ProBook-440-G8-i5-113...
1,Lenovo ThinkPad X13 Intel Core i7 12th Gen 13....,Visit the Lenovo Store,"2,00,000",,"Tomorrow, December 26",In stock.,7 days Replacement,https://www.amazon.in/Lenovo-ThinkPad-Antiglar...
2,"Lenovo V15 Intel Celeron N4020 15.6"" (39.62 cm...",Visit the Lenovo Store,"19,490",,"Tuesday, December 27",In stock.,7 days Replacement,https://www.amazon.in/Lenovo-V15-IGL-82C3A008I...
3,"HP 15s,11th Gen Intel Core i3-1115G4 8GB RAM/5...",Visit the HP Store,"41,940",,"Tomorrow, December 26",In stock.,7 days Replacement,https://www.amazon.in/HP-i3-1115G4-Micro-Edge-...
4,"Lenovo IdeaPad D330 Intel Celeron N4020 10.1"" ...",Visit the Lenovo Store,"20,034",,"Tomorrow, December 26",In stock.,7 days Replacement,https://www.amazon.in/Lenovo-IdeaPad-Detachabl...
5,"ASUS VivoBook 14 (2021), Intel Core i5-1135G7 ...",Visit the ASUS Store,"48,990",(3.7),"Tuesday, December 27",In stock.,7 days Replacement,https://www.amazon.in/ASUS-VivoBook-i5-1135G7-...
6,"HP Pavilion Plus, 12th Gen Intel Core i7 16GB ...",Visit the HP Store,"94,045",(2.4),"Tomorrow, December 26",In stock.,7 days Replacement,https://www.amazon.in/HP-Pavilion-Graphics-Bac...
7,"Honor MagicBook 15, AMD Ryzen 5 5500U 15.6-inc...",Visit the Honor Store,"39,990",(4.1),"Tomorrow, December 26",In stock.,7 days Replacement,https://www.amazon.in/Honor-MagicBook-Anti-Gla...
8,"Honor MagicBook 14, AMD Ryzen 5 5500U 14-inch ...",Visit the Honor Store,"44,990",(3.9),"Tomorrow, December 26",In stock.,7 days Replacement,https://www.amazon.in/Honor-MagicBook-Anti-Gla...
9,"Lenovo Ideapad 3 AMD Ryzen 5 5500U 15.6"" (39.6...",Visit the Lenovo Store,"44,831",(4.1),"Tomorrow, December 26",In stock.,7 days Replacement,https://www.amazon.in/Lenovo-Ideapad-Keyboard-...


Question-3

Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [13]:

import selenium
import warnings
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import requests
from selenium.webdriver.support.ui import Select

#initiating driver



s=Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=s)

driver.maximize_window()

#linking Site

driver.get("https://www.google.co.in/imghp?hl=en&tab=ri&ogbl")


#passing data in search field

#For fruits
items=driver.find_element(By.XPATH,"/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
items.send_keys("Fruits")
fruit_list=[]

#Clicking search button

search=driver.find_element(By.XPATH,"/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button")
search.click()

time.sleep(4)

for i in range (50):
    driver.execute_script("window.scrollBy(0,1000)")
    
fruits=driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")

for i in fruits:
    
    link=i.get_attribute("src")
    if link is not None:
        if link[0:4]=="http":
            fruit_list.append(link)
    

for j in range(10):
    print("Downloading Image-Number {0} of {1}".format(j+1,10))
    response=requests.get(fruit_list[j])
    gallery=open(r"F:\demo\img"+str(j)+".jpg","wb")
    gallery.write(response.content)

In [17]:
#For cars

items=driver.find_element(By.XPATH,'//*[@id="REsRA"]')
items.clear()
items.send_keys("Cars")
car_list=[]

#Clicking search button

search=driver.find_element(By.XPATH,'//*[@id="BIqFsb"]')
search.click()

time.sleep(4)

for i in range (50):
    driver.execute_script("window.scrollBy(0,1000)")
    
cars=driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")

for i in cars:
    
    link=i.get_attribute("src")
    if link is not None:
        if link[0:4]=="http":
            car_list.append(link)
    

for j in range(10):
    print("Downloading Image-Number {0} of {1}".format(j+1,10))
    response=requests.get(car_list[j])
    gallery=open(r"F:\demo\car"+str(j)+".jpg","wb")
    gallery.write(response.content)

In [23]:
#For Machine Learning

items=driver.find_element(By.XPATH,'//*[@id="REsRA"]')
items.clear()
items.send_keys("Machine Learning")

mlearn_list=[]

#Clicking search button

search=driver.find_element(By.XPATH,'//*[@id="BIqFsb"]')
search.click()

time.sleep(4)

for i in range (50):
    driver.execute_script("window.scrollBy(0,1000)")
    
ml=driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")

for i in ml:
    
    link=i.get_attribute("src")
    if link is not None:
        if link[0:4]=="http":
            mlearn_list.append(link)
    

for j in range(10):
    print("Downloading Image-Number {0} of {1}".format(j+1,10))
    response=requests.get(mlearn_list[j])
    gallery=open(r"F:\demo\ml"+str(j)+".jpg","wb")
    gallery.write(response.content)

In [21]:
#for Guitars

items=driver.find_element(By.XPATH,'//*[@id="REsRA"]')
items.clear()
items.send_keys("guitar")
guitar_list=[]

#Clicking search button

search=driver.find_element(By.XPATH,'//*[@id="BIqFsb"]')
search.click()

time.sleep(4)

for i in range (50):
    driver.execute_script("window.scrollBy(0,1000)")
    
guitar=driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")

for i in guitar:
    
    link=i.get_attribute("src")
    if link is not None:
        if link[0:4]=="http":
            guitar_list.append(link)
    

for j in range(10):
    print("Downloading Image-Number {0} of {1}".format(j+1,10))
    response=requests.get(guitar_list[j])
    gallery=open(r"F:\demo\guitar"+str(j)+".jpg","wb")
    gallery.write(response.content)

In [25]:
#For Cakes

items=driver.find_element(By.XPATH,'//*[@id="REsRA"]')
items.clear()
items.send_keys("Cake")
cakes_list=[]

#Clicking search button

search=driver.find_element(By.XPATH,'//*[@id="BIqFsb"]')
search.click()

time.sleep(4)

for i in range (50):
    driver.execute_script("window.scrollBy(0,1000)")
    
cake=driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")

for i in cake:
    
    link=i.get_attribute("src")
    if link is not None:
        if link[0:4]=="http":
            cakes_list.append(link)
    

for j in range(10):
    print("Downloading Image-Number {0} of {1}".format(j+1,10))
    response=requests.get(cakes_list[j])
    gallery=open(r"F:\demo\cake"+str(j)+".jpg","wb")
    gallery.write(response.content)

In [28]:
len(cakes_list),len(guitar_list),len(mlearn_list),len(fruit_list),len(car_list)

(76, 81, 66, 67, 69)

Question-4

Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be 
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV

In [31]:
#Importing required files

import selenium
import warnings
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


#initiating driver


s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()

#creating Empty lists
url=[]
names=[]
price=[]
color_list=[]
memory=[]
display=[]
capacity=[]
front_cam=[]
back_cam=[]

#linking Site

driver.get("https://www.flipkart.com/search?q=i+phone+14pro+max&sid=tyy%2C4io&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_2_9_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_2_9_na_na_na&as-pos=2&as-type=RECENT&suggestionId=i+phone+14pro+max%7CMobiles&requestId=1597ef4b-1889-4b36-b9b5-866847bf121d&as-searchtext=i%20phone%201")

time.sleep(7)

link=driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')

for i in link:
    locator=i.get_attribute("href")
    url.append(locator)             #getting URL

n=len(url)

try:
    
    name=driver.find_elements(By.XPATH,'//div[@class="_4rR01T"]')

    for i in name:
        nm=i.text      #getting name of mobile
        names.append(nm)

except NoSuchElemetException:
    names.append("-")

try:
    
    cost=driver.find_elements(By.XPATH,'//div[@class="_30jeq3 _1_WHN1"]')

    for i in cost:
        amt=i.text
        price.append(amt)            #getting price of mobile

except NoSuchElementException:
    price.append("-")

try:
        
    battery=driver.find_elements(By.XPATH,'//li [@class="rgWa7D"][4]')
    for j in battery:
        bat=j.text          #getting Battery details
        capacity.append(bat)

except NoSuchElementException:
    capacity.append("-")
    
try:
        
    front=driver.find_elements(By.XPATH,'//li [@class="rgWa7D"][3]')
    for j in front:
        frnt=j.text       #specifications of Primary camera
        front_cam.append(frnt)

except NoSuchElementException:
    front_cam.append("-")
try:
        
    dsize=driver.find_elements(By.XPATH,'//li [@class="rgWa7D"][2]')
    for k in dsize:
        ds=k.text             #Display size of mobile
        display.append(ds)
    
except NoSuchElementException:
        display.append("-")

try:
        
    storage=driver.find_elements(By.XPATH,'//li [@class="rgWa7D"][1]')
    for j in storage:
        mem=j.text          #Memory details(Ram/rom/extendable)
        memory.append(mem)
    
except NoSuchElementException:
    memory.append("-")
        
for i in range (n):
    
    url1=url[i]
    driver.get(url1)
    time.sleep(4)
    
    try:
        
        colour=driver.find_elements(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')
        for j in colour:
            clr=j.text   #color of mobile
            color_list.append(clr)
    
    except NoSuchElementException:
        color_list.append("-")
    
    try:
        
        back=driver.find_elements(By.XPATH,'//div[@class="_3k-BhJ"][5]/table/tbody/tr[2]/td[2]/ul/li')
        for j in back:
            bck=j.text   #back camera details
            back_cam.append(bck)
    
    except NoSuchElementException:
        back_cam.append("-")

len(memory)

24

In [21]:
df=pd.DataFrame({"Name":names,"Color":color_list,"Price":price,"Display":display,"Battery-Power":capacity,
                 "Camera":front_cam,"Link to buy":url})
df

df.to_csv('mobiles.csv')#creating data frame and csv file

In [22]:
df

,Name,Color,Price,Display,Battery-Power,Camera,Link to buy
0,"APPLE iPhone 14 Pro Max (Deep Purple, 128 GB)",Deep Purple,"₹1,39,900",17.02 cm (6.7 inch) Super Retina XDR Display,"A16 Bionic Chip, 6 Core Processor Processor",48MP + 12MP + 12MP + 12MP | 12MP Front Camera,https://www.flipkart.com/apple-iphone-14-pro-m...
1,"APPLE iPhone 14 Pro Max (Gold, 128 GB)",Gold,"₹1,39,900",17.02 cm (6.7 inch) Super Retina XDR Display,"A16 Bionic Chip, 6 Core Processor Processor",48MP + 12MP + 12MP + 12MP | 12MP Front Camera,https://www.flipkart.com/apple-iphone-14-pro-m...
2,"APPLE iPhone 14 Pro Max (Gold, 256 GB)",Gold,"₹1,49,900",17.02 cm (6.7 inch) Super Retina XDR Display,"A16 Bionic Chip, 6 Core Processor Processor",48MP + 12MP + 12MP + 12MP | 12MP Front Camera,https://www.flipkart.com/apple-iphone-14-pro-m...
3,"APPLE iPhone 14 Pro Max (Space Black, 256 GB)",Space Black,"₹1,49,900",17.02 cm (6.7 inch) Super Retina XDR Display,"A16 Bionic Chip, 6 Core Processor Processor",48MP + 12MP + 12MP + 12MP | 12MP Front Camera,https://www.flipkart.com/apple-iphone-14-pro-m...
4,"APPLE iPhone 14 Pro Max (Silver, 128 GB)",Silver,"₹1,39,900",17.02 cm (6.7 inch) Super Retina XDR Display,"A16 Bionic Chip, 6 Core Processor Processor",48MP + 12MP + 12MP + 12MP | 12MP Front Camera,https://www.flipkart.com/apple-iphone-14-pro-m...
5,"APPLE iPhone 14 Pro Max (Deep Purple, 512 GB)",Deep Purple,"₹1,69,900",17.02 cm (6.7 inch) Super Retina XDR Display,"A16 Bionic Chip, 6 Core Processor Processor",48MP + 12MP + 12MP + 12MP | 12MP Front Camera,https://www.flipkart.com/apple-iphone-14-pro-m...
6,"APPLE iPhone 14 Pro Max (Silver, 512 GB)",Silver,"₹1,69,900",17.02 cm (6.7 inch) Super Retina XDR Display,"A16 Bionic Chip, 6 Core Processor Processor",48MP + 12MP + 12MP + 12MP | 12MP Front Camera,https://www.flipkart.com/apple-iphone-14-pro-m...
7,"APPLE iPhone 14 Pro Max (Silver, 256 GB)",Silver,"₹1,49,900",17.02 cm (6.7 inch) Super Retina XDR Display,"A16 Bionic Chip, 6 Core Processor Processor",48MP + 12MP + 12MP + 12MP | 12MP Front Camera,https://www.flipkart.com/apple-iphone-14-pro-m...
8,"APPLE iPhone 14 Pro Max (Space Black, 128 GB)",Space Black,"₹1,39,900",17.02 cm (6.7 inch) Super Retina XDR Display,"A16 Bionic Chip, 6 Core Processor Processor",48MP + 12MP + 12MP + 12MP | 12MP Front Camera,https://www.flipkart.com/apple-iphone-14-pro-m...
9,"APPLE iPhone 14 Pro Max (Gold, 512 GB)",Gold,"₹1,69,900",17.02 cm (6.7 inch) Super Retina XDR Display,"A16 Bionic Chip, 6 Core Processor Processor",48MP + 12MP + 12MP + 12MP | 12MP Front Camera,https://www.flipkart.com/apple-iphone-14-pro-m...


Question-5

Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.


In [1]:
#Importing required files

import selenium
import warnings
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import re

location=input("Enter the location to get coordinates : ")

#initiating driver

s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

#linking Site

driver.get("https://www.google.com/maps")

#passing data in search field

items=driver.find_element(By.XPATH,"/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div/div[2]/form/input[1]")
items.send_keys(location)

#Clicking search button

search=driver.find_element(By.XPATH,"/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div/div[2]/div[1]/button")
search.click()

time.sleep(7)

#Fetching URL as it contains Lattitude and Longitude

geo=driver.current_url

code=geo.split("@") #Splitting Data as the data we require is after @ Symbol

#Seperating details and performing neccasary amendments in string of URL

latlong=code[1] 
lat=latlong.split(",")

lattitude=float(lat[0])
longitude=float(lat[1])

print(lattitude,longitude)
df=pd.DataFrame({"Location ":location,"Lattitude ": lattitude,"Longitude" : longitude},index=["Details"])
df

Enter the location to get coordinates : Bangalore
12.954517 77.350728


,Location,Lattitude,Longitude
Details,Bangalore,12.954517,77.350728


Question-7

Write a program to scrap all the available details of best gaming laptops from digit.in.

In [2]:
import selenium
import warnings
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#initiating driver

s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

#linking Site

driver.get("https://www.digit.in/")

#Clicking search button

laptop=driver.find_element(By.XPATH,"/html/body/div[1]/div/div[4]/ul/li[3]/a")
laptop.click()

time.sleep(5)

gaming=driver.find_element(By.XPATH,"/html/body/div[6]/div/div[2]/div[2]/ul/li[10]/a")
gaming.click()

time.sleep(6)

#Creating Array for storing data

description=[]
laptop_name=[]
full=[]

name=driver.find_elements(By.TAG_NAME,"h3")
for i in name:
    nm=i.text                                              #fetching Name of laptop
    laptop_name.append(nm)

comment=driver.find_elements(By.XPATH,"//div[@class='tptn-prod-desc']")

for i in comment:
    cmt=i.text                                             #Fetching Description
    full.append(cmt)

desc=driver.find_elements(By.XPATH,"//div[@class='Spcs-details']")

for i in desc:
    detail=i.text         #fetching specifications
    description.append(detail)

final=[]

n=len(desc)

# Applying required string functions to revove extra data

for j in range (0,n):
    temp=description[j].split("\n")
    temp1="\n".join(temp)
    final.append(temp1)
    
#creating Data frame
    
df=pd.DataFrame({"Laptop Name ":laptop_name,"Specifications ":final,"Full Description": full})
df

,Laptop Name,Specifications,Full Description
0,MSI Titan GT77-12UHS,MORE SPECIFICATIONS\nProcessor : 12th Gen Inte...,The very best MSI gaming laptop featuring an I...
1,Alienware X17 R2,MORE SPECIFICATIONS\nProcessor : 12th Gen Inte...,"Alienware, the premium gaming brand from Dell ..."
2,Acer Predator Triton 500 SE PT516-52s,MORE SPECIFICATIONS\nProcessor : 12th Gen Inte...,The Predator lineup from Acer has shown great ...
3,Omen By HP (16-B1371TX),MORE SPECIFICATIONS\nProcessor : 12th Gen Inte...,Omen by HP is a premium gaming lineup which is...
4,Acer Predator Helios 300 AN515-45,MORE SPECIFICATIONS\nProcessor : AMD Ryzen 9-5...,Acer Predator Helios 300 AN515-45 is the best ...
5,MSI Delta 15 (A5EFK-083IN),MORE SPECIFICATIONS\nProcessor : AMD 5th Gen R...,The best MSI gaming laptop with an AMD process...
6,Omen By HP (16-C0141AX),MORE SPECIFICATIONS\nProcessor : AMD Ryzen™ 9 ...,Another worthy gaming laptop from the Omen lin...
7,Lenovo Legion 5i Pro (82RF00MGIN),MORE SPECIFICATIONS\nProcessor : 12th Gen Inte...,Lenovo has gone a little mainstream with their...
8,Alienware M15 R5 Ryzen Edition Icc-C780001win,MORE SPECIFICATIONS\nProcessor : AMD Ryzen R7-...,Dell not only has gaming laptops under the Del...
9,Lenovo Slim 7 Gen 6 (82K8002JIN),MORE SPECIFICATIONS\nProcessor : Lenovo Ryzen ...,Lenovo has been pushing for thinner gaming lap...


In [3]:
for i in range (1,10):
    print(final[i])

MORE SPECIFICATIONS
Processor : 12th Gen Intel Core i9-12900H 14 core processor with 5 GHz clock speed
Display : 17.3″ (1920 x 1080) screen, 360 Hz refresh rate
OS : Windows 11 Home
Memory : 32 GB DDR5 RAM & 1 TB SSD
Graphics Processor : 16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics card
Body : 299 x 359 x 21 mm dimension & 2.96 kg weight
Price : ₹ 389,990
MORE SPECIFICATIONS
Processor : 12th Gen Intel Core i7-12700H 14 core processor with 3.5 GHz clock speed
Display : 16″ (2560 x 1600) screen
OS : Windows 11
Memory : 32 GB DDR5 RAM & 2 TB SSD
Graphics Processor : 8 GB DDR6 NVIDIA GeForce RTX 3070 Ti Graphics card
Body : 262 x 359 x 20 mm dimension & 2.4 kg weight
Price : ₹ 229,990
MORE SPECIFICATIONS
Processor : 12th Gen Intel Core i7-12700H 14 core processor with 4.7 GHz clock speed
Display : 16.1″ (2560 x 1440) screen
OS : Windows 11 Home
Memory : 8 GB DDR5 RAM & 1 TB SSD
Graphics Processor : 8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card
Body : 369 x 248 x 23 mm dimension & 2.32

In [4]:
for i in range(1,10):
    print(full[i])

Alienware, the premium gaming brand from Dell has its own reputation for building some of the meanest gaming machines for more than two decades. The best Alienware gaming laptop featuring an Intel processor is the Alienware x17 R2. This thing packs an Intel Core i9-12900HK which is a 14-Core (6P+8E) processor which has 20 Threads. The lower P-Core count helps it hit slightly higher clock speeds but the Max Turbo power is capped at 115W which is less than what the MSI GT77 sports. As for the graphics card, the Alienware x17 R2 also sports an NVIDIA RTX 3080 Ti with the peak TGP set to 175 Watts. The CPU+GPU combination of the Alienware x17 R2 comes close to the MSI GT77. As for the memory, the Alienware x17 R2 ships with 32 GB of DDR5 RAM clocked at 4800 MT/s. The display on the x17 R2 is a 17.3-inch FHD 360 Hz panel compared to the 120 Hz 4K panel on the MSI GT77. You’ll end up seeing higher frame rates on the Alienware x17 R2 without any tearing whereas you’ll see crisper visuals at h

Question-8

Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be 
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry

In [2]:
import selenium
import warnings
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#initiating driver

s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

driver.maximize_window()

#Creating Arrays for storage

rank=[]
names=[]
age_list=[]
country=[]
category=[]
source=[]
net_worth=[]

#linking Site

driver.get("https://www.forbes.com")

time.sleep(3)

#Clicking Main Menu

search=driver.find_element(By.XPATH,"/html/body/div[1]/header/nav/div[1]/div/div/div[1]")
search.click()

time.sleep(4)

#Clicking Billionare tab

billionare=driver.find_element(By.XPATH,"/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[1]/a")
billionare.click()

time.sleep(3)

#Getting Billionare List

billionareList=driver.find_element(By.XPATH,"/html/body/div[1]/main/div/section/section[1]/div/div/div[1]/div/div[1]/div[1]/div[2]/a/h2")
billionareList.click()

time.sleep(4)

for page in range (1,14):          #Clicking next page untill the next button is active
        
    try:
        position=driver.find_elements(By.XPATH,"//div[@Class='rank']")
        for i in position:
            pos=i.text                 #Fetching Positions
            rank.append(pos)
    
    except NoSuchElementException:
        rank.append("-")
    
    try:
        name=driver.find_elements(By.XPATH,"//div[@Class='personName']")
        for i in name:
            nm=i.text
            names.append(nm)                       #Fetching Names
    
    except NoSuchElementException:
        names.append("-")
    
    try:
        age=driver.find_elements(By.XPATH,"//div[@Class='age']")
        for i in age:
            ag=i.text
            age_list.append(ag)                                 #Fetching Age
    
    except NoSuchElementException:
        age_list.append("-")
    
    try:    
        worth=driver.find_elements(By.XPATH,"//div[@Class='netWorth']")
        for i in worth:
            nw=i.text
            net_worth.append(nw)                                #Fetching NetWorth
    
    except NoSuchElementException:
        net_worth.append("-")
        
    try:
        
        place=driver.find_elements(By.XPATH,"//div[@Class='countryOfCitizenship']")
        for i in place:
            ctry=i.text                                       #Fetching Country
            country.append(ctry)
    
    except NoSuchElementException:
        country.append("-")
    
    try:
        src=driver.find_elements(By.XPATH,"//div[@Class='source']")
        for i in src:
            sr=i.text                                  #Fetching Source
            source.append(sr)
    
    except NoSuchElementException:
        source.append("-")
    
    try:
        
        division=driver.find_elements(By.XPATH,"//div[@Class='category']")
        for i in division:
            ctgry=i.text
            category.append(ctgry)                           #Fetching Division details
    
    except NoSuchElementException:
        category.append("-")
        
    
       
    next_page=driver.find_element(By.XPATH,"//div[@class='next-page']" ) 
    next_page.click()
    time.sleep(5)

          #Clicking next page untill the next button is active
        
#Creating Data Frame

billionares=pd.DataFrame({"Rank ":rank,"Name ":names,"Age":age_list,"Country":country,
                     "Category":category,"Source" :source ,"Net-Worth" : net_worth,
                     
                    })
billionares

,Rank,Name,Age,Country,Category,Source,Net-Worth
0,1.,Elon Musk,50,United States,Automotive,"Tesla, SpaceX",$219 B
1,2.,Jeff Bezos,58,United States,Technology,Amazon,$171 B
2,3.,Bernard Arnault & family,73,France,Fashion & Retail,LVMH,$158 B
3,4.,Bill Gates,66,United States,Technology,Microsoft,$129 B
4,5.,Warren Buffett,91,United States,Finance & Investments,Berkshire Hathaway,$118 B
...,...,...,...,...,...,...,...
2595,2578.,Jorge Gallardo Ballart,80,Spain,Healthcare,pharmaceuticals,$1 B
2596,2578.,Nari Genomal,82,Cyprus,Fashion & Retail,apparel,$1 B
2597,2578.,Ramesh Genomal,71,Cyprus,Fashion & Retail,apparel,$1 B
2598,2578.,Sunder Genomal,68,United Kingdom,Fashion & Retail,garments,$1 B


Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted
from any YouTube Video.

In [1]:
from youtube_comment_scraper_python import *
import pandas as pd

comments=[]

youtube.open("https://www.youtube.com/watch?v=shz8Uxo4gJw")
for i in range(0,10):
    response = youtube.video_comments()

    cmt = response['body']
    comments.extend(cmt)

Details=pd.DataFrame(comments)
Details

Progress:  66%|█████████████████████████████████████████████                       | 132.6/200 [01:10<01:22,  1.23s/it]
Progress:   0%|                                                                                | 0/200 [00:00<?, ?it/s]
Progress:  84%|█████████████████████████████████████████████████████████▍          | 168.8/200 [01:18<00:10,  3.09it/s]
Progress: 100%|██████████████████████████████████████████████████████▉| 199.60000000000002/200 [01:20<00:00,  5.29it/s]
Progress:  38%|██████████████████████████▌                                          | 77.0/200 [00:08<00:12,  9.60it/s]
Progress: 212.00000000000003it [01:33,  2.45it/s]                                                                      
Progress: 213.40000000000003it [01:35,  2.17it/s]                                   | 89.4/200 [00:20<00:42,  2.59it/s]
Progress: 214.80000000000004it [01:37,  1.88it/s]                                   | 90.8/200 [00:22<00:48,  2.24it/s]
Progress: 216.20000000000005it [01:39,  

,Comment,Likes,Time,UserLink,user
0,saviours of humanity,24K,9 days ago,https://www.youtube.com/channel/UC2PvMOTaTWZMk...,
1,i am not a IAS aspirant but i have a lot of re...,867,9 days ago,NaN,
2,"""Overthinking is the real problem of today's y...",835,6 days ago,NaN,
3,"NO ADS, NO SONG, NO DANCE, No paid Promotion, ...",517,7 days ago,NaN,
4,First Khan sir then Vikas Divyakriti sir now t...,137,4 days ago,NaN,
...,...,...,...,...,...
855,Great work sir ️,2,5 days ago,NaN,
856,A personality like Mathematics Guru RK Srivast...,0,7 days ago,NaN,
857,The toughest job in the world is telling the t...,0,3 days ago,NaN,
858,I love this person seriously man ️ he is very ...,1,7 days ago,NaN,


Progress: 1357.3999999999853it [37:51,  1.67s/it]


Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews,
overall reviews, privates from price, dorms from price, facilities and property description.

In [1]:
#Question-10

import selenium
import warnings
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.ui import Select

#initiating driver



s=Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=s)


driver.maximize_window()
#Creating Empty arrays

price=[]
distance=[]
names=[]
ratings=[]
reviews=[]
private=[]
dorm=[]
facilities=[]
description=[]
overall=[]
links=[]

#linking Site

driver.get("https://www.hostelworld.com/s?q=London,%20England&country=England&city=London&type=city&id=3&from=2022-12-21&to=2022-12-24&guests=2&page=1")

time.sleep(3)

while True:
    
    try:
        
        name=driver.find_elements(By.XPATH,"//h2[@Class= 'title title-6']")

        for i in name:
            nm=i.text                 #Fetching Names
            names.append(nm)
    
    except NoSuchElementException:
        names.append("-")
        

    try:
        
        dist=driver.find_elements(By.XPATH,"//span[@Class= 'description']")

        for i in dist:
            dis=i.text               #Fetching Distance from city center
            distance.append(dis.replace("Hostel - ",""))
    
    except NoSuchElementException:
        distance.append("-")

    try:
        
        score = driver.find_elements(By.XPATH,"//div[@Class= 'score orange big' or @Class= 'score gray big']")
        for i in score:
            rate=i.text            #Fetching Ratings
            ratings.append(rate)
    
    except NoSuchElementException:
        ratings.append("-")

    try:
        
        over_all=driver.find_elements(By.XPATH,"//div[@Class= 'keyword']")
        for i in over_all:
            rate1=i.text
            overall.append(rate1)  #Fetching overall rating
    except NoSuchElementException:
        over_all.append("-")

    try:
        
        review=driver.find_elements(By.XPATH,"//div[@Class= 'reviews']")

        for i in review:
            total=i.text            #getting Reviews
            reviews.append(total)  
    
    except NoSuchElementException:
        reviews.append("-")

    try:
        pprice=driver.find_elements(By.XPATH,"//div[@Class= 'price-col']")

        for i in pprice:
            pp=i.text       #getting Prices
            price.append(pp) 
    
    except NoSuchElementException:
        price.append("-")
    
    n=len(price)

    for i in range(0,n):             #Seperating price of private room and dormatory
        if i%2==0:
            private.append(price[i].replace("\n"," "))
        else:
            dorm.append(price[i].replace("\n"," "))

    link=driver.find_elements(By.XPATH,"//a[@Class= 'view-button']")
    
    for i in link:
        desc=i.get_attribute("href")      #Getting link of hostels
        links.append(desc)
    
    try:
        
        next_page=driver.find_element(By.XPATH,'//div[@class="pagination-item pagination-next"]')
        next_page.click()
        time.sleep(4)
    
    except:
        break
    
    
total=len(links)

for j in range (0,total) :

    url=links[j]
    driver.get(url)
    time.sleep(2)
    try:
        
        details=driver.find_elements(By.XPATH,"/html/body/div[3]/div/div/div[2]/section/div[6]/div/div[2]/div/div/div[2]")
        for m in details:
            desc=m.text             #fetching proprty description
            description.append(desc)
    
    except NoSuchElementException:
        description.append("-")
        
    try:
        
        facility=driver.find_elements(By.XPATH,"//ul[@class='facility-sections']")

        for k in facility:
            det=k.text               #fetching available facilities
            facilities.append(det.replace("\n",": "))
    
    except NoSuchElementException:
        facilities.append("-")
 

   #creating dataframe
    
hostel=pd.DataFrame({"Hostel-Name ":names,"Distance ":distance,"Ratings":ratings,"Total-Reviews":reviews,
                     "Private-room cost":private,"Dormatry Charges" :dorm ,"Facilities Available" : facilities,
                     "Overall-Rating":overall[2:],"Property-Description":description
                    })



In [10]:
len(distance),len(names),len(ratings),len(reviews),len(private),len(dorm),len(facilities),len(description),len(overall),len(links)

(30, 30, 30, 30, 30, 30, 30, 30, 32, 30)

In [2]:
hostel

,Hostel-Name,Distance,Ratings,Total-Reviews,Private-room cost,Dormatry Charges,Facilities Available,Overall-Rating,Property-Description
0,Wombat's City Hostel London,3.6km from city centre,8.9,14420 Total Reviews,Privates From Rs13939.76 Rs11152,Dorms From Rs2974.42 Rs2380,Free: Linen Included Free City Maps Free WiFi ...,Fabulous,A safe haven in the middle of the metropolis: ...
1,St Christopher's Village,1.8km from city centre,8.0,11770 Total Reviews,No Privates Available,Dorms From Rs1652.04 Rs1487,Free: Linen Included Free City Maps Free WiFi ...,Fabulous,COVID 19 Policy Update.\nIn response to Corona...
2,Hostel One Notting Hill,5.5km from city centre,9.3,1643 Total Reviews,Privates From Rs12033,Dorms From Rs2493,Free: Linen Included Free WiFi Free Internet A...,Superb,The perfect place for solo travelers to connec...
3,Astor Hyde Park,4.3km from city centre,8.7,11814 Total Reviews,Privates From Rs8787,Dorms From Rs2197,Free: Linen Included Free City Maps Free WiFi ...,Fabulous,Located in the most beautiful and charming nei...
4,Urbany Hostel London,5.4km from city centre,9.2,577 Total Reviews,Privates From Rs9175,Dorms From Rs2502,Free: Linen Included Free WiFi: General: Secur...,Superb,"Welcome to Urbany Hostel London, our first int..."
5,Generator London,3km from city centre,7.7,7297 Total Reviews,Privates From Rs6201,Dorms From Rs2094,Free: Linen Included Free City Maps Free WiFi ...,Very Good,Generator London is a design hotel-hostel loca...
6,Safestay London Elephant & Castle,1.7km from city centre,7.3,4736 Total Reviews,Privates From Rs18873,Dorms From Rs4224,Free: Linen Included Free City Maps Free WiFi:...,Very Good,Safestay at Elephant & Castle is ideal if you ...
7,Smart Russell Square Hostel,2.6km from city centre,7.1,9848 Total Reviews,Privates From Rs10501,Dorms From Rs1937,Free: Linen Included Free City Maps Free WiFi ...,Very Good,Smart Hostels is proud to present our most cen...
8,St Christopher's Camden,4.3km from city centre,8.2,3931 Total Reviews,No Privates Available,Dorms From Rs1872.83 Rs1686,Free: Linen Included Free City Maps Free WiFi ...,Fabulous,COVID 19 Policy Update.\nIn response to Corona...
9,Smart Hyde Park Inn Hostel,5km from city centre,7.5,6452 Total Reviews,Privates From Rs12319.89 Rs11088,Dorms From Rs1807.38 Rs1627,Free: Linen Included Free City Maps Free WiFi:...,Very Good,Smart Hyde Park Inn Hostel is perfectly locate...


In [3]:
facilities

['Free: Linen Included Free City Maps Free WiFi Free Internet Access Free City Tour: General: Wheelchair Friendly Security Lockers Key Card Access Common Room Elevator Bicycle Parking BBQ Breakfast Not Included Meeting Rooms Adaptors Book Exchange Hot Showers Self-Catering Facilities Dishwasher Cooker Fridge/Freezer Utensils Dryer Outdoor Terrace Reading Light Hair Dryers For Hire Iron / Ironing Board Safe Deposit Box Microwave Washing Machine Towels Not Included Follows Covid-19 sanitation guidance: Services: Internet access Laundry Facilities Towels for hire Luggage Storage Fax Service 24 Hour Reception ATM Housekeeping Internet café Late check-out: Food & Drink: Bar Meals Available Vending Machines Tea & Coffee Making Facilities: Entertainment: Board Games Pool Table Foosball Wi-Fi',
 'Free: Linen Included Free City Maps Free WiFi Free Internet Access: General: Security Lockers Key Card Access Common Room Elevator Breakfast Not Included Air Conditioning Adaptors Book Exchange Hot Sh

In [4]:
description

["A safe haven in the middle of the metropolis: We took a traditional brick building with roots going back to the 1800s and set a new standard for hostel life in the UK.\nWhat used to be sailors' lodgings in the olden days is now a hostel satisfying all needs of the modern traveler. With great attention to detail, we have created a hostel in London that celebrates the legacy of our historic building.\nOld oak that was previously part of sturdy ships and churches is now immortalized in our furniture throughout the house. The pints you drink here will be served on a bar made from 500-year-old wood. \n\nBut as comfortable as our hostel might be, London's calling - and London's call is hard to ignore, given our location: \nOur Wombat's City Hostel is situated in the London Borough of Tower Hamlets, right next to famous Tower Bridge and Tower of London, charming markets, authentic pubs, and plenty of other historic sights.\nA short walk will see you in the trendy Borough of Shoreditch with 